In [ ]:
import pandas as pd

def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}
    
    from pandas import DataFrame
    from urllib.parse import urlencode
    import json
    
    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(DataFrame(d, columns=h))

In [ ]:
## get all teams
params = {'LeagueID':'00'}
teams = get_nba_data('commonTeamYears', params)
teams[teams.MAX_YEAR == '2017'] #take out teams no longer active
teams.head()

In [ ]:
## get all players
params = {'LeagueID':'00', 'Season': '2016-17' , 'IsOnlyCurrentSeason': '0'}
players = get_nba_data('commonallplayers', params)
players = players[players.TO_YEAR == '2017'] #players that are currently active 
players = players[players.TEAM_ID != 0]
players.tail()

In [ ]:
#get all team names
allteam_dd_text = players.TEAM_CITY+' '+players.TEAM_NAME
allteam_dd = dict(zip(allteam_dd_text, players.TEAM_ID))
allteam_dd

In [ ]:
plyr_by_team_dd = dict()
for t, p in players.groupby('TEAM_ID'):
    plyr_by_team_dd[t] = dict(zip(p.DISPLAY_LAST_COMMA_FIRST, p.PERSON_ID))

plyr_dd_text = players.DISPLAY_LAST_COMMA_FIRST
plyr_dd_id = players.PERSON_ID
plyr_dd = dict(zip(plyr_dd_text, plyr_dd_id))
plyr_dd

In [ ]:
params = {'LeagueID': '00',
                'Season' : '2007-08',
                'IsOnlyCurrentSeason' : '0'}
player_year = get_nba_data('commonallplayers', params)
player_year = player_year[player_year.TEAM_ID != 0]
player_year[player_year.TEAM_ID == 1610612738]

In [ ]:
from ipywidgets import interact, FloatSlider, Dropdown, Button
import ipywidgets as widgets

year = widgets.Text(
        value='2016-17',
        placeholder='Format: NNNN-NN (eg. 1995-96)',
        description='Season Year',
        disabled=False
        )

def update_year(change):
    params = {'LeagueID': '00',
                'Season' : year.value,
                'IsOnlyCurrentSeason' : '0'}
    player_year = get_nba_data('commonallplayers', params)
    player_year = player_year[player_year.TEAM_ID != 0] #takes out all players not affiliated
    #creates dictionary based on all players in this year
    plyr_by_team_dd = dict()
    for t, p in player_year.groupby('TEAM_ID'):
        plyr_by_team_dd[t] = dict(zip(p.DISPLAY_LAST_COMMA_FIRST, p.PERSON_ID))
    #organizes dictionary
    plyr_dd_text = player_year.DISPLAY_LAST_COMMA_FIRST
    plyr_dd_id = player_year.PERSON_ID
    plyr_dd = dict(zip(plyr_dd_text, plyr_dd_id))
    selected = "Golden State Warriors"
    team_menu = Dropdown(options=allteam_dd, label = selected)
    player_menu = Dropdown(options=plyr_by_team_dd[allteam_dd[selected]])
    def update_team(change):
        player_menu.options = plyr_by_team_dd[change['new']]
    display(team_menu, player_menu, season_type, player_stats) #players in entered season year
    team_menu.observe(update_team, names='value') #update players accordingly by choosing which team

populate = Button(description = 'Populate Players', icon = 'check')
populate.on_click(update_year)

season_type = widgets.RadioButtons(
                options=['Pre Season', 'Regular Season', 'Playoffs'],
                description='Season Type',
                disabled=False
                )
print ("Playoffs 2017-18 not updated on NBA website")
display(year, populate)

player_stats = Button(description = 'Player Stats', icon = 'check')
#player_stats.on_click(calculate)

def calculate(stats):
    params = {'TeamID' : team_menu.value,
              'Season' : year.value,
              'SeasonType' : season_type.value}
    game_log = get_nba_data('teamgamelog', params)    

    

In [ ]:
params_team1 = {'TeamID' : '1610612744',
              'Season' : '2017-18',
              'SeasonType' : 'Regular Season'}
team_log = get_nba_data('teamgamelog', params_team1)
team_log.columns.values

In [ ]:
params_player = {'PlayerID' : '201939',
              'Season' : '2017-18',
              'SeasonType' : 'Regular Season'}
player_log = get_nba_data('playergamelog', params_player)
player_log.columns.values
#fgm, fga, fg_pct, fg3m, fg3a, fg3_pct, ftm, fta, ft_pct, oreb, dreb, reb, ast, stl, blk, tov, pf, pts

- FGM: Field Goals Made
- FGA: Field Goals attempted
- FG_PCT: Field Goals Percentage
- FG3M: 3-pointers Made
- FG3A: 3-pointers Attempted
- FG3_PCT: 3-pointers Percentage
- FTM: Free Throws Made
- FTA: Free Throws Attempted
- FT_PCT: Free Throws Percentage
- OREB: Offensive Rebounds
- DREB: Defensive Rebounds
- REB: Rebounds
- AST: Assists
- STL: Steals
- BLK: Blocks
- TOV: Turnovers
- PF: Personal Fouls
- PTS: Points

We will only be displaying FGM, OREB, DREB, AST, STL, BLK, TOV, PTS